## Objective
* Gather the data using the Twitter API.
* Store and maintain a PostgreSQL database.
* Create a live dashboard that pulls from the database and analyzes the sentiment of each tweet.

## Background Information
* Cyberpunk 2077 is an upcoming action role-playing video game developed by CD Projekt. With such a large social media following, users are voicing their wishes and concerns to CD Projekt. In this project, we will monitor tweets collected and parse through the sentiment of each. 

## Process:
* Data Gathering
* PostgreSQL database
* Dashboard



## Table of Contents:
* Part I: Data Gathering
    * Gathering
    * PostgreSQL
* Part II: Dashboard


In [ ]:
# Import packages
from datetime import datetime
from dash.dependencies import Input, Output
from io import BytesIO
from sqlalchemy import create_engine
from sqlalchemy_utils import database_exists, create_database
from tweepy import API
from tweepy import OAuthHandler
from tweepy import Stream
from urllib3.exceptions import ProtocolError
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer
from wordcloud import WordCloud

import base64
import dash
import dash_table
import dash_core_components as dcc
import dash_html_components as html
import matplotlib.pyplot as plt
import pandas as pd
import plotly.graph_objects as go
import psycopg2
import tweepy

# PART I - Data Gathering

First, we'll need to pull the data from the twitter API using tweepy.

In [ ]:
# Twitter API credentials
consumer_key = "Enter"
consumer_secret = "Enter"
access_key = "Enter"
access_secret = "Enter"
auth = OAuthHandler(consumer_key, consumer_secret)
auth.set_access_token(access_key, access_secret)
api = tweepy.API(auth)

# Pull tweets from today's date, we'll start with 300 tweets.
tweets = tweepy.Cursor(api.search, q = ['#Cyberpunk'],
                       since = datetime.now().strftime('%Y-%m-%d'), lang = "en",
                       tweet_mode = 'extended').items(330)

# Store the tweets in a list
tweet_list = [tweet for tweet in tweets]

# Create a dataframe to store the contents of the tweets
tweet_text = []
for x in tweet_list:
    tweet_text.append(x.full_text)

df_tweets = pd.DataFrame(data = tweet_text, columns = ['tweet'])

Next, we'll store the dataframe into our PostgreSQL database.

In [ ]:
# PostgreSQL
## Connect to our database
DATABASE_URL = "Enter"
engine = create_engine(DATABASE_URL)
conn = psycopg2.connect(DATABASE_URL, sslmode = 'require')
cursor = conn.cursor()

## Drop existing tables with the tweets name
cursor.execute('DROP TABLE tweets')
conn.commit()

## Store all the tweets from our dataframe to our database
df_tweets.to_sql('tweets', con = engine)


# PART II - Dashboard

In [ ]:
################################ DASHBOARD ##################################
# Functions / Variables that need to be assigned for the dashboard

## Pull the data from the database
### Set up the connection
DATABASE_URL = "Enter"
conn = psycopg2.connect(DATABASE_URL, sslmode = 'require')

### Store into our dataframe df
df = pd.read_sql('select * from tweets', con = conn, index_col = 'index')

### Reindex the values (we will use these for our twitter feed)
df_1t = df[0:30].reset_index()
df_2t = df[31:61].reset_index()
df_3t = df[62:92].reset_index()
df_4t = df[93:123].reset_index()
df_5t = df[124:154].reset_index()
df_6t = df[155:185].reset_index()
df_7t = df[186:216].reset_index()
df_8t = df[217:247].reset_index()
df_9t = df[248:278].reset_index()
df_10t = df[279:309].reset_index()

## Dataframe that will contain all the contents and sentiment of the tweets.
total_tweets_df = pd.DataFrame(columns = ['Tweets', 'Sentiment'])

## Vader Sentiment Analyzer
analyser = SentimentIntensityAnalyzer()


## Interval tracker for live updating
def get_value(df, n_intervals):
    """Function which outputs the tweet text given the interval for live updates"""
    text = df['tweet'][n_intervals]
    return text

## Vader sentiment analyzer
def sentiment_analyzer_scores(sentence):
    """Function which returns the sentiment polarity score from VADER"""
    score = analyser.polarity_scores(sentence)
    return score


## Sentiment class definition
def sentiment_logic(text):
    """Function which outputs the sentiment class."""
    if text >= 0.05:
        result = 'Positive'
    elif (text > -0.05) & (text < 0.05):
        result = 'Neutral'
    elif text <= -0.05:
        result = 'Negative'
    return result


#--------------------- Datatable
def datatable_asset(df):
    """Function to create a datatable which is used to return the tweets and sentiment."""
    datatable = dash_table.DataTable( 
        id = 'typing_formatting_1',
        data = df.to_dict('records'),
        columns =
        [
            {
                'id': 'Tweets',
                'name': 'Tweet',
                'type': 'text'
            }, 

            {
                'id': 'Sentiment',
                'name': 'Sentiment',
                'type': 'text'
            }, 

            


        ],
        
        # Highlight Cells based on conditions - first, second, and third row
        style_data_conditional =
        [
            # Highlighting sentiment analysis results
            {
                "if": {"column_id": "Sentiment",
                       "filter_query": "{Sentiment} = Positive"},
                "backgroundColor": "#a6f1a6",
                'color': 'black'
            },

            {
                "if": {"column_id": "Sentiment",
                       "filter_query": "{Sentiment} = Negative"},
                "backgroundColor": "#ff0000",
                'color': 'black'
            },

            {
                "if": {"column_id": "Sentiment",
                       "filter_query": "{Sentiment} = Neutral"},
                "backgroundColor": "#e0e0e0",
                'color': 'black'
            },
            # Fix columnd widths
            {'if': {'column_id': 'Tweets'},
             'width': '90%'},
            
            {'if': {'column_id': 'Sentiment'},
             'width': '10%'},
        ],
        
        # Formatting the data/headers cells
        style_cell = {'backgroundColor': '#f7f7f7', 'font-family': 'helvetica',
                      'fontColor': '#000000', 'fontSize': 24, 'textAlign': 'left'
                     },

        style_data = {'border': '1px solid #00a8ff', 'font-size': 24,
                      'font-family': 'helvetica', 'whiteSpace': 'normal', 
                     },

        style_header = {'border': '1px solid #00a8ff', 'font-size': 28,
                        'font-family': 'helvetica', 'textAlign': 'center',
                        'fontWeight': 'bold'
                       },

        css = [{
            'selector': '.dash-spreadsheet td div',
            'rule': '''
            line-height: 35px;
            max-height: 70px; min-height: 70px; height: 70px;
            display: block;
            overflow-y: hidden;
            '''
        }],
        
        tooltip_data = [{
            column: {'value': str(value), 'type': 'markdown'}
            for column, value in row.items()
        }
            for row in df.to_dict('rows')
        ],
        
        tooltip_duration = None,
        editable = True,
        page_size = 10,
        filter_action = "native",
        sort_action = "native",
        sort_mode = "multi",
        column_selectable = "single",
        row_selectable = "multi",
        row_deletable = True,
        selected_columns = [],
        selected_rows = [],
        page_action = "native",
        
    )
    return datatable


## Sentiment pie graphs
def piegraph(df):
    """Function which returns the pie graphs used for sentiment classification in the twitter feed."""
    fig = go.Figure(data = [
        go.Pie(
            labels = list(df.keys()), values = list(df.values()),
            textinfo = 'label+percent', insidetextorientation = 'radial'
        )
    ])
    
    fig.update_layout(paper_bgcolor = '#eaeaea', height = 550,
                      width = 550, font_size = 20,
                      uniformtext_minsize = 20, uniformtext_mode = 'hide',
                      hoverlabel = dict(font_size = 24)
                     )
    return fig


## Word Cloud
def plot_word_cloud(text):
    """Function to create and plot a worcloud"""

    # The regex expression is used to eliminate all non english letters
    regex_expression = r"[a-zA-Z]+"
    
    # Word Cloud
    wc = WordCloud(width = 800, height = 600, max_words = 10000, 
                      relative_scaling = 0, background_color = '#f7f7f7', contour_color = "black",
                      regexp = regex_expression, random_state = 2, colormap = 'gnuplot2',
                      collocations = False,
             ).generate(text)
    
    wc_img = wc.to_image()
    with BytesIO() as buffer:
        wc_img.save(buffer, 'png')
        final_img = base64.b64encode(buffer.getvalue()).decode()
    
    return final_img


## Sentiment Distribution
def plot_histogram(df):
    """Function which returns a distribution of sentiment classes"""
    colors = ["#a6f1a6", "#e0e0e0", "#ff0000"]
    fig = go.Figure(data=[
        go.Histogram(x = df, marker_color = colors)
    ])
    
    fig.update_xaxes(linewidth = 1, linecolor = 'black', 
                     gridcolor = 'LightPink', automargin = True,  
                     ticks = "outside", tickwidth = 2,
                     tickcolor = 'black', ticklen = 12,
                     title = 'Sentiment', title_font = dict(size = 22))
    
    fig.update_yaxes(linewidth = 1, linecolor = 'black', 
                     gridcolor = 'LightPink', ticks = "outside",
                     tickwidth = 2, tickcolor = 'black',
                     ticklen = 12, title = 'Frequency',
                     title_font = dict(size = 22),
                    )
    
    fig.update_layout(
        font = dict(size = 18),
        legend = dict(
            x = 1,
            y = 1,
            traceorder = "normal",
            font = dict(
                family = "sans-serif",
                size = 18,
                color = "black"
            ),
            bgcolor = "#f7f7f7",
            bordercolor = "#f7f7f7",
            borderwidth = 1
        ),
        plot_bgcolor = "#f7f7f7", paper_bgcolor = "#f7f7f7",
        width = 900, height = 600, 
        hoverlabel = dict(
            font_size = 24, 
            font_family = "Rockwell"),
        xaxis = {'categoryorder':'category descending'}
    )
    return fig


## CSS applied to the dashboard
external_css = [
    
    # Normalize the CSS
    "https://cdnjs.cloudflare.com/ajax/libs/normalize/7.0.0/normalize.min.css",
    
    # Fonts
    "https://fonts.googleapis.com/css?family=Open+Sans|Roboto",
    "https://maxcdn.bootstrapcdn.com/font-awesome/4.7.0/css/font-awesome.min.css",
    '/assets/base-styles.css',
    '/assets/custom-styles.css',
]


## Dash application
app = dash.Dash(__name__, external_stylesheets = external_css)

app.layout = html.Div([
    html.Div([
                html.Div(
                    [
                        
                        # Input Title of Dashboard, Include title and href link
                        html.H2(
                            id = "banner-title",
                            children = [
                                html.A(
                                    "Cyberpunk Twitter Sentiment Analysis",
                                    href = "https://github.com/SulmanK/Cyberpunk-2077-Twitter-Sentiment-Analysis",
                                    style = {"text-decoration": "none", "color": "inherit", 'padding-left': '55rem'},
                                )
                            ] 
                        ), 
                       
                        # Insert Github Logo with href link
                        html.A(
                            [
                                html.Img(src = app.get_asset_url("github_logo.png"))
                            ], href = "https://github.com/SulmanK/Cyberpunk-2077-Twitter-Sentiment-Analysis",
                        ),
                        
                        # Insert Dash logo with href link
                        html.A( 
                            [
                                html.Img(src = app.get_asset_url("dash_banner.png")) 
                            ], href = "https://dash.plotly.com/",
                        ),
                        
                        # Insert Cyberpunk logo with href link
                        html.A( 
                            [
                                html.Img(src = app.get_asset_url("cyberpunk_logo.png")) 
                            ], href = "https://www.cyberpunk.net/us/en/",
                        ), 
                        
                    ], className = "row",
                )
            ], className = "banner"
),


## Insert Project Introduction
        html.Div(
            [
                dcc.Markdown(
                    ''' 
Cyberpunk 2077 is an upcoming action role-playing video game developed by CD Projekt. With such a large social media following, users are voicing their wishes and concerns to CD Projekt. In this project, we will monitor tweets collected and parse through the sentiment of each. More information on the database collection and sentiment analysis tools used assumptions are in the project repository page and notebook.
'''
                )
            ], style = {'padding': '2rem 4rem 2rem 4rem',
                        'border-top': '10px solid #2DDBE8',
                        'border-bottom': '10px solid #2DDBE8',
                        'fontSize' : 28, 'font-family': "Myriad Pro"}
        ),  
    
## Insert Twitter Feed
    html.Div([
        html.H2('Twitter Feed')
    ]),
    
## Storing our interval in this component every 30 seconds it updates.
    html.Div([
        html.Div(id = 'tweets'),
        dcc.Interval(
            id = 'interval-component',
            interval = 30 * 1000, # in milliseconds
            n_intervals = 0,
        )
    ]),   

# Insert Exploration Section (Word Cloud and Distribution Plot)
    html.Div([
        html.H2('Exploration')
    ]),
    
    html.Div(id = 'Exploration')
    
 ], style = {'maxWidth': '2000px', 'height': '80vh',
             'minWidth': '1500px', 'padding-left': '20px'
            }   
)




## Callback function for updating the twitter feed cycles using the intervals

@app.callback(Output('tweets', 'children'),
              [Input('interval-component', 'n_intervals')])
def update_tweets_feed(n):
    """Function which is used to update the twitter feed."""
    
    # Retrieve the tweets
    first_tweet = get_value(df_1t, n)
    second_tweet = get_value(df_2t, n) 
    third_tweet = get_value(df_3t, n)
    fourth_tweet = get_value(df_4t, n)
    fifth_tweet = get_value(df_5t, n)
    sixth_tweet = get_value(df_6t, n)
    seventh_tweet = get_value(df_7t, n)
    eighth_tweet = get_value(df_8t, n)
    nineth_tweet = get_value(df_9t, n)
    tenth_tweet = get_value(df_10t, n) 
    
    # Compute the sentiment of each tweet
    sa_first_tweet = sentiment_analyzer_scores(first_tweet)
    sa_second_tweet = sentiment_analyzer_scores(second_tweet)
    sa_third_tweet = sentiment_analyzer_scores(third_tweet)
    sa_fourth_tweet = sentiment_analyzer_scores(fourth_tweet)
    sa_fifth_tweet = sentiment_analyzer_scores(fifth_tweet)
    sa_sixth_tweet = sentiment_analyzer_scores(sixth_tweet)
    sa_seventh_tweet = sentiment_analyzer_scores(seventh_tweet)
    sa_eighth_tweet = sentiment_analyzer_scores(eighth_tweet)
    sa_nineth_tweet = sentiment_analyzer_scores(nineth_tweet)
    sa_tenth_tweet = sentiment_analyzer_scores(tenth_tweet)
    
    # Return the tweet contents and a pie graph of the sentiment.
    
    return html.Div([
        html.Div([

# First Tweet
            html.Div([
                html.Div([
                    html.Pre(str(first_tweet)),
                ], 
                    className = 'ten columns',
                    style = {
                        'backgroundColor': 'white',
                        'box-shadow': '2px 2px 10px #ccc',
                        'padding': '10px',
                        'padding-bottom': '25px',
                        'margin': '30px',
                        'overflowX': 'scroll',
                        'fontSize': '22px',
                    }
                ),
                html.Div([
                    dcc.Graph(figure = piegraph(sa_first_tweet))
                ],
                    className = 'nine columns',
                    style = {"padding-left": "550px", }
                ),
            ], 
                className = 'row' 
            ),
 
# Second Tweet
            
            html.Div([
                html.Div([
                    html.Pre(str(second_tweet)),
                ], 
                    className = 'ten columns',
                    style = {
                        'backgroundColor': 'white',
                        'box-shadow': '3px 3px 10px #ccc',
                        'padding': '10px',
                        'padding-bottom': '25px',
                        'margin': '30px',
                        'overflowX': 'scroll',
                        'fontSize': '22px'}
                ),
                html.Div([
                    dcc.Graph(figure = piegraph(sa_second_tweet))
                ],
                    className = 'nine columns',
                    style = {"padding-left": "550px"}
                ),
            ], 
                className = 'row' 
            ),
        
 # Third Tweet
            
            html.Div([
                html.Div([
                    html.Pre(str(third_tweet)),
                ], 
                    className = 'ten columns',
                    style = {
                        'backgroundColor': 'white',
                        'box-shadow': '3px 3px 10px #ccc',
                        'padding': '10px',
                        'padding-bottom': '25px',
                        'margin': '30px',
                        'overflowX': 'scroll',
                        'fontSize': '22px'}
                ),
                html.Div([
                    dcc.Graph(figure = piegraph(sa_third_tweet))
                ],
                    className = 'nine columns',
                    style = {"padding-left": "550px"}
                ),
            ], 
                className = 'row' 
            ),
        
 # Fourth Tweet
        
        html.Div([
            html.Div([
                html.Pre(str(fourth_tweet)),
            ], 
                className = 'ten columns',
                style = {
                    'backgroundColor': 'white',
                    'box-shadow': '3px 3px 10px #ccc',
                    'padding': '10px',
                    'padding-bottom': '25px',
                    'margin': '30px',
                    'overflowX': 'scroll',
                    'fontSize': '22px'}
            ),
            html.Div([
                dcc.Graph(figure = piegraph(sa_fourth_tweet))
            ],
                className = 'nine columns',
                style = {"padding-left": "550px"}
            ),
        ], 
            className = 'row' 
        ),


 # Fifth Tweet
        
        html.Div([
            html.Div([
                html.Pre(str(fifth_tweet)),
            ], 
                className = 'ten columns',
                style = {
                    'backgroundColor': 'white',
                    'box-shadow': '3px 3px 10px #ccc',
                    'padding': '10px',
                    'padding-bottom': '25px',
                    'margin': '30px',
                    'overflowX': 'scroll',
                    'fontSize': '22px'}
            ),
            html.Div([
                dcc.Graph(figure = piegraph(sa_fifth_tweet))
            ],
                className = 'nine columns',
                style = {"padding-left": "550px"}
            ),
        ], 
            className = 'row' 
        ),
        

 # Sixth Tweet
        html.Div([
            html.Div([
                html.Pre(str(sixth_tweet)),
            ], 
                className = 'ten columns',
                style = {
                    'backgroundColor': 'white',
                    'box-shadow': '3px 3px 10px #ccc',
                    'padding': '10px',
                    'padding-bottom': '25px',
                    'margin': '30px',
                    'overflowX': 'scroll',
                    'fontSize': '22px'}
            ),
            html.Div([
                dcc.Graph(figure = piegraph(sa_sixth_tweet))
            ],
                className = 'nine columns',
                style = {"padding-left": "550px"}
            ),
        ], 
            className = 'row' 
        ),
        
 # Seventh Tweet
        
        html.Div([
            html.Div([
                html.Pre(str(seventh_tweet)),
            ], 
                className = 'ten columns',
                style = {
                'backgroundColor': 'white',
                'box-shadow': '3px 3px 10px #ccc',
                'padding': '10px',
                'padding-bottom': '25px',
                'margin': '30px',
                'overflowX': 'scroll',
                'fontSize': '22px'}
            ),
    
    html.Div([
        dcc.Graph(figure = piegraph(sa_seventh_tweet))
    ],
        className = 'nine columns',
        style = {"padding-left": "550px"}
    ),
        ], 
            className = 'row' 
        ),

 # Eighth Tweet
        
        html.Div([
            html.Div([
                html.Pre(str(eighth_tweet)),
            ], 
                className = 'ten columns',
                style = {
                    'backgroundColor': 'white',
                    'box-shadow': '3px 3px 10px #ccc',
                    'padding': '10px',
                    'padding-bottom': '25px',
                    'margin': '30px',
                    'overflowX': 'scroll',
                    'fontSize': '22px'}
                    ),
    
    html.Div([
        dcc.Graph(figure = piegraph(sa_eighth_tweet))
    ],
        className = 'nine columns',
        style = {"padding-left": "550px"}
    ),
        ], 
            className = 'row' 
        ),

 # Nineth
        
        html.Div([
            html.Div([
                html.Pre(str(nineth_tweet)),
            ], 
                className = 'ten columns',
                style = {
                    'backgroundColor': 'white',
                    'box-shadow': '3px 3px 10px #ccc',
                    'padding': '10px',
                    'padding-bottom': '25px',
                    'margin': '30px',
                    'overflowX': 'scroll',
                    'fontSize': '22px'}
            ),
            html.Div([
                dcc.Graph(figure = piegraph(sa_nineth_tweet))
            ],
                className = 'nine columns',
                style = {"padding-left": "550px"}
            ),
        ], 
            className = 'row' 
        ),

 # Tenth Tweet
        
        html.Div([
            html.Div([
                html.Pre(str(tenth_tweet)),
            ], 
                className = 'ten columns',
                style = {
                    'backgroundColor': 'white',
                    'box-shadow': '3px 3px 10px #ccc',
                    'padding': '10px',
                    'padding-bottom': '25px',
                    'margin': '30px',
                    'overflowX': 'scroll',
                    'fontSize': '22px'}
            ),
            html.Div([
                dcc.Graph(figure = piegraph(sa_tenth_tweet))
            ],
                className = 'nine columns',
                style = {"padding-left": "550px"}
            ),
        ], 
            className = 'row' 
        ),
        ], style = {'overflowY': 'scroll', 'overflowX': 'hidden',
                    'maxHeight': '105ex', 'backgroundColor' : '#eaeaea'}
    ),
    
    ])




# Exploration callback functions
@app.callback(Output('Exploration', 'children'),
              [Input('interval-component', 'n_intervals')])

def exploration(n):
    "Function which returns the contents of the exploration section - wordcloud and sentiment distribution"
    # Retrieve the tweet contents
    first_tweet = get_value(df_1t, n)
    second_tweet = get_value(df_2t, n) 
    third_tweet = get_value(df_3t, n)
    fourth_tweet = get_value(df_4t, n)
    fifth_tweet = get_value(df_5t, n)
    sixth_tweet = get_value(df_6t, n)
    seventh_tweet = get_value(df_7t, n)
    eighth_tweet = get_value(df_8t, n)
    nineth_tweet = get_value(df_9t, n)
    tenth_tweet = get_value(df_10t, n) 
    
    # Sentiment of each tweet
    sa_first_tweet = sentiment_analyzer_scores(first_tweet)
    sa_second_tweet = sentiment_analyzer_scores(second_tweet)
    sa_third_tweet = sentiment_analyzer_scores(third_tweet)
    sa_fourth_tweet = sentiment_analyzer_scores(fourth_tweet)
    sa_fifth_tweet = sentiment_analyzer_scores(fifth_tweet)
    sa_sixth_tweet = sentiment_analyzer_scores(sixth_tweet)
    sa_seventh_tweet = sentiment_analyzer_scores(seventh_tweet)
    sa_eighth_tweet = sentiment_analyzer_scores(eighth_tweet)
    sa_nineth_tweet = sentiment_analyzer_scores(nineth_tweet)
    sa_tenth_tweet = sentiment_analyzer_scores(tenth_tweet)
    
    # Compute the compound score for obtaining a sentiment class
    compound_score_first_tweet = sentiment_logic((list(sa_first_tweet.values())[list(sa_first_tweet.keys()).index('compound')] ))
    compound_score_second_tweet = sentiment_logic((list(sa_second_tweet.values())[list(sa_second_tweet.keys()).index('compound')] )) 
    compound_score_third_tweet = sentiment_logic((list(sa_third_tweet.values())[list(sa_third_tweet.keys()).index('compound')] ))
    compound_score_fourth_tweet = sentiment_logic((list(sa_fourth_tweet.values())[list(sa_fourth_tweet.keys()).index('compound')] ))
    compound_score_fifth_tweet = sentiment_logic((list(sa_fifth_tweet.values())[list(sa_fifth_tweet.keys()).index('compound')] ))
    compound_score_sixth_tweet = sentiment_logic((list(sa_sixth_tweet.values())[list(sa_sixth_tweet.keys()).index('compound')] ))
    compound_score_seventh_tweet = sentiment_logic((list(sa_seventh_tweet.values())[list(sa_seventh_tweet.keys()).index('compound')] ))
    compound_score_eighth_tweet = sentiment_logic((list(sa_eighth_tweet.values())[list(sa_eighth_tweet.keys()).index('compound')] ))
    compound_score_nineth_tweet = sentiment_logic((list(sa_nineth_tweet.values())[list(sa_nineth_tweet.keys()).index('compound')] ))
    compound_score_tenth_tweet = sentiment_logic((list(sa_tenth_tweet.values())[list(sa_tenth_tweet.keys()).index('compound')] ))
    
    # Create a new temporary dataframe for the tweet contents and sentiment
    compound_score_list = [compound_score_first_tweet, compound_score_second_tweet,
                           compound_score_third_tweet, compound_score_fourth_tweet,
                           compound_score_fifth_tweet, compound_score_sixth_tweet, 
                           compound_score_seventh_tweet, compound_score_eighth_tweet,
                           compound_score_nineth_tweet, compound_score_tenth_tweet]
    
    
    first_col = [first_tweet, second_tweet,
              third_tweet, fourth_tweet,
              fifth_tweet, sixth_tweet,
              seventh_tweet, eighth_tweet,
              nineth_tweet, tenth_tweet]
    
    second_col = compound_score_list
    
    tmp_df = pd.DataFrame(data = {'Tweets' : first_col, 
                                 'Sentiment' : second_col})
   
    # total_tweets_df will be dataframe used
    global total_tweets_df
    
    # Append new rows from the tmp_df
    total_tweets_df = total_tweets_df.append(tmp_df)
    
    # Extract the contents of the tweets for the wordcloud
    full_string = ''
    for x in total_tweets_df['Tweets']:
        full_string += x
    
    wc = plot_word_cloud(full_string)
    
    # Initialize datatable
    datatable = datatable_asset(df = total_tweets_df)
    
    # Initialize histogram
    histogram = plot_histogram(total_tweets_df['Sentiment'])
    
    return html.Div([
        # Datatable
        html.Div([
            datatable
        ], className = 'row'
        ),
       # Wordcloud and Distribution
        html.Div([
            html.Div([
                html.Img(src="data:image/png;base64," + wc)
            ],
                className = 'one-half column'
            ),
            html.Div([
                dcc.Graph(figure = histogram)
            ],
                className = 'one-half column')             
        ],
            className = 'row'
        )
    ])
        
        

if __name__ == '__main__':
    app.run_server(debug=False)